In [1]:
import numpy as np
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
# Select CPU/GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('Device:', device)

Device: cuda


In [3]:
# Generate a fake data set
input_dim = 64
targets = [np.asarray([mean, stdev]) for mean in np.linspace(-49, 50, 100) for stdev in np.linspace(1, 100, 100)]
images = [np.random.normal(mean, stdev, size=(1, input_dim, input_dim)) for mean, stdev in targets]
assert len(targets) == len(images)

In [4]:
# Split the data into train, validation, test
X_trv,   X_test, Y_trv,   Y_test = train_test_split(images, targets, train_size=0.8)
X_train, X_val,  Y_train, Y_val  = train_test_split(images, targets, train_size=0.75)


class MyDataset(Dataset):
    
    def __init__(self, X, Y):
        super().__init__()
        self.X = X
        self.Y = Y

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.from_numpy(self.Y[idx])
    
    def __len__(self):
        return len(self.X)


# Convert to Pytorch data loaders
batch_size = 100
training   = DataLoader(MyDataset(X_train, Y_train), batch_size=batch_size)
validation = DataLoader(MyDataset(X_val,   Y_val),   batch_size=batch_size)
test       = DataLoader(MyDataset(X_test,  Y_test),  batch_size=batch_size)

In [5]:
# Define our CNN
class CNN(nn.Module):

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(input_dim*input_dim*2, 2)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [6]:
# Train the CNN
num_epochs = 5
learning_rate = 0.001

# Model, loss, and optimizer
cnn = CNN()
cnn = cnn.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(cnn.parameters(), lr=learning_rate)

# Train the Model
for epoch in tqdm(range(num_epochs), desc='Epochs', leave=True):
    for i, (images, targets) in enumerate(tqdm(training, desc='Batches', leave=False)):
        images, targets = images.to(device, dtype=torch.float), targets.to(device, dtype=torch.float)
        outputs = cnn(images)              # Forward
        loss = criterion(outputs, targets) # Compute loss
        optimizer.zero_grad()              # Clear stored gradients
        loss.backward()                    # Compute gradients
        optimizer.step()                   # Update weights

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Iter [{}/{}] Loss: {:6.4f}'.format(
                (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item())))

In [ ]:
### Test the Model ###
# Change model to 'eval' mode
# This affects the batch normalization layer, so that it uses the mean/variance obtained in training
cnn.eval()  

# Compute accuracy
correct = 0
total = 0

for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy (10000 test images): %d %%' % (100 * correct / total))

In [ ]:
r2  = r2_score(Y_true_unscaled, Y_pred_unscaled)
mae = mean_absolute_error(Y_true_unscaled, Y_pred_unscaled)